In [1]:
from osgeo import gdal, gdalconst
import numpy as np
from os import listdir
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
driver = gdal.GetDriverByName("HFA")
driver.Register()

6

# load feature files from folder

#load data by block

In [3]:
def load_data_block(filepath,m):
    filename_list = listdir(filepath)
    f = []
    for filename in filename_list:
        if filename.endswith(".dat"):
            f.append(filepath+"\\"+filename)
    dsbyblock = []
    for feature_file in f:
        ds = gdal.Open(feature_file)
        cols = ds.RasterXSize
        rows = ds.RasterYSize
        bands = ds.RasterCount
        for i in range(0, rows, 1):
            if i + m <= rows:
                for j in range(0, cols, 1):
                    if j + m <= cols:
                        data = ds.ReadAsArray(j, i, m, m).reshape(m*m*4,)/10000
                        dsbyblock.append(data)
    return dsbyblock

In [4]:
h = ["Shrub", "Masson's pine","Other broadleaf","Bare soil","Chinese fir"]
dsbyblock_ss = []
blocks_labels = []
m = 11
j = 1
for i in h:
    a = load_data_block(r"G:\Paper\python\TensorFlow\test04\dataset"+"\\"+i,m)
    dsbyblock_ss.extend(a)
    b = np.ones([len(a),])*j
    blocks_labels = np.hstack((blocks_labels,b))
    j = j + 1
    print(i+" nums: "+ str(len(a)))
print(len(dsbyblock_ss))
print(blocks_labels, blocks_labels.shape)

brush nums: 918
maweisong nums: 1089
qitakuoye nums: 1053
road nums: 968
shamu nums: 1366
5394
[ 1.  1.  1. ...,  5.  5.  5.] (5394,)


In [5]:
# #m = 7 
# brush nums: 1350
# maweisong nums: 1625
# qitakuoye nums: 1525
# road nums: 1400
# shamu nums: 2150
# 8050
##m = 5
# brush nums: 1602
# maweisong nums: 1941
# qitakuoye nums: 1797
# road nums: 1652
# shamu nums: 2614
# 9606
##m = 9
# brush nums: 1122
# maweisong nums: 1341
# qitakuoye nums: 1277
# road nums: 1172
# shamu nums: 1734
# 6646
# m =11
# brush nums: 918
# maweisong nums: 1089
# qitakuoye nums: 1053
# road nums: 968
# shamu nums: 1366
# 5394

# one hot for labels dataset

labels for block datasets

In [6]:
onehot_encoder = OneHotEncoder(sparse=False)
blocks_labels_onehot = onehot_encoder.fit_transform(blocks_labels.reshape(-1, 1))
print(blocks_labels_onehot)
print(blocks_labels_onehot.shape)

[[ 1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 ..., 
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]]
(5394, 5)


# make datasets

In [7]:
blocks_dataset = {}
blocks_dataset['feature'] = np.array(dsbyblock_ss)
blocks_dataset['target'] = blocks_labels_onehot
blocks_dataset['target name'] = h

In [8]:
print(blocks_dataset)

{'feature': array([[ 0.059 ,  0.058 ,  0.0573, ...,  0.4484,  0.4367,  0.4294],
       [ 0.058 ,  0.0573,  0.0579, ...,  0.4367,  0.4294,  0.4434],
       [ 0.0573,  0.0579,  0.0584, ...,  0.4294,  0.4434,  0.4567],
       ..., 
       [ 0.0494,  0.0476,  0.0479, ...,  0.3437,  0.3407,  0.3388],
       [ 0.0476,  0.0479,  0.0477, ...,  0.3407,  0.3388,  0.3448],
       [ 0.0479,  0.0477,  0.0465, ...,  0.3388,  0.3448,  0.3577]]), 'target': array([[ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.]]), 'target name': ['brush', 'maweisong', 'qitakuoye', 'road', 'shamu']}


In [9]:
x_train, x_validation, y_train, y_validation =train_test_split(blocks_dataset["feature"], blocks_dataset["target"],
                                                   random_state=0)

In [10]:
print(x_train.shape, y_train[1],y_train.shape)
print(x_validation.shape, y_validation[1],y_validation.shape)

(4045, 484) [ 0.  0.  0.  0.  1.] (4045, 5)
(1349, 484) [ 0.  0.  0.  1.  0.] (1349, 5)


# TensorFlow for CNN

In [11]:
# make batches
batch_size = 10
n_batch = x_train.shape[0] // batch_size
# define function
def variable_summaries(var):
    with tf.name_scope("summaries"):
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean)
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar("stddev", stddev)
        tf.summary.scalar("max", tf.reduce_max(var))
        tf.summary.scalar("min", tf.reduce_min(var))
        tf.summary.histogram("histogram", var)
# init weights function
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# init biases function
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# define convo layer function
def conv2d(x,W):
    # x input tensor of shape [batch, in_height, in_width, in_channels]
    # W filter/kernel tensor of shape [filter_height, filter_width, in_channels,out_channels]
    #strides[0]=strides[3]=1,strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding: A "string" from:"SAME", "VALID"
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")
def conv3d(x,W):
    # x input tensor of shape [batch, in_depth,in_height, in_width, in_channels]
    # W filter/kernel tensor of shape [filter_depth,filter_height, filter_width, in_channels,out_channels]
    #strides[0]=strides[4]=1,strides[1]代表x方向的步长，strides[2]代表y方向的步长,strides[3]代表z方向的步长
    #padding: A "string" from:"SAME", "VALID"
    return tf.nn.conv3d(x, W, strides=[1, 1, 1, 1,1], padding="SAME")
# define pooling layer
def max_pool_2x2(x):
    #ksize(1, x, y, 1)
    return tf.nn.max_pool(x,ksize=[1, 2, 2, 1], strides= [1, 2, 2, 1], padding="SAME")


with tf.name_scope("input"):
# make placeholder
    x = tf.placeholder(tf.float32, [None,m*m*4], name="x")
    y = tf.placeholder(tf.float32, [None,5], name="y")
    x_image = tf.reshape(x, [-1,m,m, 4])
with tf.name_scope("conv1"):
    # initial first conv layer weights and biases
    with tf.name_scope("W_conv1"):
        W_conv1 = weight_variable([3, 3, 4, 16])
        variable_summaries(W_conv1)
    with tf.name_scope("b_conv1"):
        b_conv1 = bias_variable([16])
        variable_summaries(b_conv1)
    # x_image and weights biases  conv1, relu activation function
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
with tf.name_scope("conv2"):
    # initial second conv layer weights and biases
    with tf.name_scope("W_conv2"):
        W_conv2 = weight_variable([3, 3, 16, 32])
        variable_summaries(W_conv2)
    with tf.name_scope("b_conv2"):
        b_conv2 = bias_variable([32])
        variable_summaries(b_conv2)
# h_pool1 and weights biases  conv2, relu activation function
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2) 
with tf.name_scope("fc1"):
    # initial first full-connectelly layer weights and biases
    with tf.name_scope("W_fc1"):
        W_fc1 = weight_variable([3*3*32, 1000])
        variable_summaries(W_fc1)
    with tf.name_scope("b_fc1"):
        b_fc1 = bias_variable([1000])
        variable_summaries(b_fc1)
#convert pool layer into 1D
    h_pool2_flat = tf.reshape(h_pool2,[-1, 3*3*32])

# get first full-connect layer output
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
with tf.name_scope("dropout"):
# keep_prob and dropout
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

with tf.name_scope("softmax"):
# initial second full-connect layer weights and biases
    with tf.name_scope("W_softmax"):
        W_fc2 = weight_variable([1000, 5])
        variable_summaries(W_fc2)
    with tf.name_scope("b_softmax"):
        b_fc2 = bias_variable([5])
        variable_summaries(b_fc2)
# finally output
    prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)
# cross-entropy
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
    tf.summary.scalar("loss",loss)
with tf.name_scope("train"):   
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
with tf.name_scope("accuracy"): 
# accuracy
    with tf.name_scope("correct_prediction"):  
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    with tf.name_scope("accuracy"):  
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy",accuracy)
# initial variables
init = tf.global_variables_initializer()
saver = tf.train.Saver()
# make session execu
merge = tf.summary.merge_all()
train_acc_summary = tf.summary.scalar("train_accuracy",accuracy)
validation_acc_summary = tf.summary.scalar("validation", accuracy)
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter(r"G:\Paper\python\TensorFlow\test06\logfile/cnn_11x11lr=1000=1e-4-2c_kp==0.5", sess.graph)
    best_validation_accuracy = 0.0
    last_improvement = 0
    require_improvement = 100
    total_iterations = 0
    for epoch in range(1000):
        total_iterations += 1
        for i in range(n_batch):
            batch_x = x_train[i*batch_size:(i+1)*batch_size]
            batch_y = y_train[i*batch_size:(i+1)*batch_size]
            summary,_ = sess.run([merge,train_step], feed_dict={x:batch_x, y:batch_y, keep_prob:0.5})
        train_acc_str = sess.run(train_acc_summary, feed_dict={x:x_train, y:y_train, keep_prob:1.0})
        validation_acc_str = sess.run(validation_acc_summary,feed_dict={x:x_validation, y:y_validation, keep_prob:1.0})
        writer.add_summary(train_acc_str,epoch)
        writer.add_summary(validation_acc_str, epoch)
        writer.add_summary(summary,epoch)
        if (total_iterations % 10 == 0) or (epoch == 999):
            train_acc = sess.run(accuracy, feed_dict={x:x_train, y:y_train, keep_prob:1.0})
            validation_acc = sess.run(accuracy,feed_dict={x:x_validation, y:y_validation, keep_prob:1.0})
            if validation_acc > best_validation_accuracy:
                best_validation_accuracy = validation_acc
                last_improvement = total_iterations
                saver.save(sess,r"G:\Paper\python\TensorFlow\test06\cnn_11x11lr=1000=1e-4-2c_kp==0.5/my_net.ckpt")
                improved_str = "*"
            else:
                improved_str = " "
            msg = "Iter: {0:>6}, Train Accuracy: {1:>6.1%}, Validation Acc: {2:>6.1%} {3}"
            print(msg.format(epoch + 1, train_acc, validation_acc, improved_str))
        if total_iterations - last_improvement > require_improvement:
            print("No improvement found in a while, stopping optimization")
            break

Iter:     10, Train Accuracy:  97.7%, Validation Acc:  96.7% *
Iter:     20, Train Accuracy:  98.1%, Validation Acc:  97.7% *
Iter:     30, Train Accuracy:  98.3%, Validation Acc:  98.1% *
Iter:     40, Train Accuracy:  97.9%, Validation Acc:  97.7%  
Iter:     50, Train Accuracy:  98.9%, Validation Acc:  99.0% *
Iter:     60, Train Accuracy:  99.0%, Validation Acc:  99.1% *
Iter:     70, Train Accuracy:  99.0%, Validation Acc:  99.1%  
Iter:     80, Train Accuracy:  99.1%, Validation Acc:  99.2% *
Iter:     90, Train Accuracy:  99.2%, Validation Acc:  99.3% *
Iter:    100, Train Accuracy:  99.7%, Validation Acc:  99.9% *
Iter:    110, Train Accuracy:  99.6%, Validation Acc:  99.5%  
Iter:    120, Train Accuracy:  99.6%, Validation Acc:  99.3%  
Iter:    130, Train Accuracy:  99.8%, Validation Acc:  99.5%  
Iter:    140, Train Accuracy: 100.0%, Validation Acc:  99.9% *
Iter:    150, Train Accuracy: 100.0%, Validation Acc: 100.0% *
Iter:    160, Train Accuracy: 100.0%, Validation Acc: 1

In [12]:
# cnn-5 kp=0.5 390 1.00

In [13]:
#cnn-7 kp=0.5  380 1.0

In [14]:
# cnn-9 kp=0.5 290  1.0

In [15]:
# cnn-11 kp=0.5  260 1.0